# Analiza orzeczeń sądów powszechnych

In [1]:
import spacy 
import pandas as pd
import numpy as np
import os
os.chdir('..')

In [ ]:
#https://www.kaggle.com/code/sauravmaheshkar/auto-completion-using-n-gram-models

In [ ]:
# etykietuj sądy - okręgowe // apelacyjne

In [6]:
import spacy
nlp = spacy.load('pl_core_news_sm')
dflist = []
for i in range(1,10):
    df = pd.read_csv('./resources/compensation_with_lemmas_chunk_{}.zip'.format(i))
    dflist.append(df)

data = pd.concat(dflist)
del dflist
# df = df[['id', 'courttype', 'judgmenttype', 'textcontent']]
# df = df[df["courttype"] == 'COMMON'] 
# df = df[df["judgmenttype"] == 'SENTENCE']
# df['reasoning'] = df['text'].str.extract('(uzasadnienie[^>]*)', expand=True)
# df['sentence'] = df['text'].str.extract('([^>]*uzasadnienie)', expand=True)
# df['text'] = df.textcontent.str.replace(r'<[^<>]*>', '', regex=True)
# df.text = df.text.replace(r'\n',' ', regex=True) 
# df.text = df['text'].str.lower()
# df["lemmatized"] = df.reasoning.apply(lambda x: " ".join([y.lemma_ for y in nlp(str(x))]))

In [9]:
#df_test['kwoty'] = df_test['text'].str.extract('(zasądza (...) na rzecz powoda kwotę [\d+.]*)', expand=True)
df.sentence = df.sentence.str.replace('\.','', regex=True)
#replace spaces between numbers
df.sentence = df.sentence.str.replace('(\d)\s+(\d)','', regex=True)
# replace spaces between numbers in text
df['kwoty'] = df['sentence'].str.extract('(zasąd(?:za|ił)(?:\W+\w+){0,5}?\W+na rzecz (?:powoda|powódki|strony powodowej) kwotę [\d+.]*)', expand=True)
# zasądza / zasądził od pozwanego na rzecz powoda kwotę
# na rzecz powódki
#todo
df_kwoty = df[~df['kwoty'].isnull()]

In [12]:
df

In [68]:
def findSum(str1):
 
    # A temporary string
    temp = "0"
 
    # holds sum of all numbers
    # present in the string
    Sum = 0
 
    # read each character in input string
    for ch in str1:
 
        # if current character is a digit
        if (ch.isdigit()):
            temp += ch
 
        # if current character is an alphabet
        else:
 
            # increment Sum by number found
            # earlier(if any)
            Sum += int(temp)
 
            # reset temporary string to empty
            temp = "0"
 
    # atoi(temp.c_str1()) takes care
    # of trailing numbers
    return Sum + int(temp)

In [71]:
df_kwoty['target'] = df_kwoty['kwoty'].apply(lambda x: findSum(str(x)))

<ipython-input-71-0fe16bfd95b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kwoty['target'] = df_kwoty['kwoty'].apply(lambda x: findSum(str(x)))


In [73]:
import spacy
nlp = spacy.load('pl_core_news_sm')
df_kwoty["lemmatized"] = df_kwoty['uzasadnienie'].apply(lambda x: " ".join([y.lemma_ for y in nlp(str(x))]))
X = df_kwoty['lemmatized']
stopwords = ['protokolant','sędzia','ssa', 'spr','a','aby','ach','acz','aczkolwiek','aj','albo','ale','alez','ależ','ani','az','aż','bardziej','bardzo','beda','bedzie','bez','deda','będą','bede','będę','będzie','bo','bowiem','by','byc','być','byl','byla','byli','bylo','byly','był','była','było','były','bynajmniej','cala','cali','caly','cała','cały','ci','cie','ciebie','cię','co','cokolwiek','cos','coś','czasami','czasem','czemu','czy','czyli','daleko','dla','dlaczego','dlatego','do','dobrze','dokad','dokąd','dosc','dość','duzo','dużo','dwa','dwaj','dwie','dwoje','dzis','dzisiaj','dziś','gdy','gdyby','gdyz','gdyż','gdzie','gdziekolwiek','gdzies','gdzieś','go','i','ich','ile','im','inna','inne','inny','innych','iz','iż','ja','jak','jakas','jakaś','jakby','jaki','jakichs','jakichś','jakie','jakis','jakiś','jakiz','jakiż','jakkolwiek','jako','jakos','jakoś','ją','je','jeden','jedna','jednak','jednakze','jednakże','jedno','jego','jej','jemu','jesli','jest','jestem','jeszcze','jeśli','jezeli','jeżeli','juz','już','kazdy','każdy','kiedy','kilka','kims','kimś','kto','ktokolwiek','ktora','ktore','ktorego','ktorej','ktory','ktorych','ktorym','ktorzy','ktos','ktoś','która','które','którego','której','który','których','którym','którzy','ku','lat','lecz','lub','ma','mają','mało','mam','mi','miedzy','między','mimo','mna','mną','mnie','moga','mogą','moi','moim','moj','moja','moje','moze','mozliwe','mozna','może','możliwe','można','mój','mu','musi','my','na','nad','nam','nami','nas','nasi','nasz','nasza','nasze','naszego','naszych','natomiast','natychmiast','nawet','nia','nią','nic','nich','nie','niech','niego','niej','niemu','nigdy','nim','nimi','niz','niż','no','o','obok','od','około','on','ona','one','oni','ono','oraz','oto','owszem','pan','pana','pani','po','pod','podczas','pomimo','ponad','poniewaz','ponieważ','powinien','powinna','powinni','powinno','poza','prawie','przeciez','przecież','przed','przede','przedtem','przez','przy','roku','rowniez','również','sam','sama','są','sie','się','skad','skąd','soba','sobą','sobie','sposob','sposób','swoje','ta','tak','taka','taki','takie','takze','także','tam','te','tego','tej','ten','teraz','też','to','toba','tobą','tobie','totez','toteż','totobą','trzeba','tu','tutaj','twoi','twoim','twoj','twoja','twoje','twój','twym','ty','tych','tylko','tym','u','w','wam','wami','was','wasz','wasza','wasze','we','według','wiele','wielu','więc','więcej','wlasnie','właśnie','wszyscy','wszystkich','wszystkie','wszystkim','wszystko','wtedy','wy','z','za','zaden','zadna','zadne','zadnych','zapewne','zawsze','ze','zeby','zeznowu','zł','znow','znowu','znów','zostal','został','żaden','żadna','żadne','żadnych','że','żeby']

<ipython-input-73-fbae37056a10>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kwoty["lemmatized"] = df_kwoty['uzasadnienie'].apply(lambda x: " ".join([y.lemma_ for y in nlp(str(x))]))


In [87]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
ylabels = df_kwoty['target']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)
bow = CountVectorizer(stop_words=stopwords, ngram_range=(6,6), min_df=60, max_df=600)
chi2_selector = SelectKBest(chi2, k=500)
#clf = LinearRegression(fit_intercept=False)
#clf = RandomForestRegressor(max_depth=2, random_state=0)
#('poly', PolynomialFeatures(degree=3)),
clf = ElasticNet(random_state=0)
pipel = Pipeline([('bow', bow),
                  ('chi2_feature_selection', chi2_selector),
                 ('classifier', clf)])

pipel.fit(X_train, y_train)
print("Accuracy Score:",pipel.score(X_test, y_test))
import eli5
features = eli5.show_weights(clf, vec=bow, top=1000)

Accuracy Score: 0.015697856522995712


In [86]:
features

Weight,Feature
0.1506 ± 0.4514,ciało rozstrój zdrowie postać np zeszpecenia
0.1191 ± 0.3205,mieścić ramy szkoda normalny związek przyczynowy
0.1149 ± 0.4311,000 tytuł zadośćuczynienie wraz ustawowymi odsetka
0.0333 ± 0.2137,koszt sądowy sprawa cywilny dz 2014
0.0326 ± 0.1940,dzień złożenie zawiadomienie szkoda chyba ustalenie
0.0275 ± 0.2723,dzień liczyć dzień złożenie poszkodowany uprawniony
0.0240 ± 0.2350,móc żądać odsetka opóźnienie chociażby ponieść
0.0210 ± 0.2439,mechaniczny obowiązany odszkodowanie wyrządzić związek rucho
0.0201 ± 0.2106,możliwy pociągać zobowiązać nadmierny trudność koszt
0.0199 ± 0.2263,mógłby osiągnąć szkoda wyrządzić przesłanka odpowiedzialność


In [81]:
from sklearn import metrics

y_test_pred = pipel.predict(X_test)
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2))

Mean absolute error = 11936.4
Mean squared error = 1386862801.07
Median absolute error = 6608.79
Explain variance score = -0.03
R2 score = -0.03
